In [1]:
import pandas as pd
import geopandas as gpd
import os
from dash import Dash, html, dcc, Input, Output

from handlers.general_visualization import GeneralVisualization
from handlers.district_visualization import DistrictVisualization
from handlers.metro_visualization import MetroVisualization
from handlers.crime_visualization import CrimeVisualization
from handlers.general_use import add_borders

In [ ]:
# Importar datasets
data_path = os.path.join(os.path.dirname(os.getcwd()),  'data', 'pancho_clean')
listings = pd.read_csv(os.path.join(data_path, 'listings_merged.csv'))
calendar = pd.read_csv(os.path.join(data_path, 'calendar.csv'))
gdf = gpd.read_file(os.path.join(data_path, 'neighbourhoods.geojson'))
crimes = pd.read_csv(os.path.join(data_path, 'crimes.csv'))
metro = pd.read_csv(os.path.join(data_path, 'metro.csv'))

# Mario
mario_data_path = os.path.join(os.path.dirname(os.getcwd()),  'data', 'mario_clean')
fuentes_mascotas = pd.read_csv(os.path.join(mario_data_path, 'fuentes_mascotas.csv'))
fuentes_potables = pd.read_csv(os.path.join(mario_data_path, 'fuentes_potables.csv'))
jardines = pd.read_csv(os.path.join(mario_data_path, 'jardines.csv'))
restaurantes = pd.read_csv(os.path.join(mario_data_path, 'restaurantes.csv'))
salud = pd.read_csv(os.path.join(mario_data_path, 'salud.csv'))

In [3]:
gv = GeneralVisualization(listings, gdf)
dv = DistrictVisualization(listings, gdf)
mv = MetroVisualization(listings, metro, gdf)
cv = CrimeVisualization(crimes, gdf)

In [4]:
crimes.columns

Index(['DISTRITOS', 'RELACIONADAS CON LAS PERSONAS',
       'RELACIONADAS CON EL PATRIMONIO', 'POR TENENCIA DE ARMAS',
       'POR TENENCIA DE DROGAS', 'POR CONSUMO DE DROGAS'],
      dtype='object')

In [5]:
distritos = list(listings['neighbourhood_group'].unique())

app = Dash(__name__, assets_folder='assets')

app.layout = html.Div(className='container', children=[
	html.H1('Datos de AirBnB en Madrid'),
	
	dcc.Tabs(children=[
		dcc.Tab(label='Información general', children=[
			html.Div(id='general-graphs')
		]),

		dcc.Tab(label='Información por distrito', children=[
			html.H4('Selecciona un distrito:'),
			dcc.Dropdown(
				id='distrito-dropdown-cloropleth',
				options=[{'label': distrito, 'value': distrito} for distrito in distritos],
				value='CENTRO'
			),
			html.Div(id='distrito-graph-cloropleth', className='map-container')
		]),

		dcc.Tab(label='Información con metro', children=[
            html.Div(id='metro-graphs')
		]),

		dcc.Tab(label='Crímenes', children=[
			html.H4('Selecciona un crimen:'),
			dcc.Dropdown(
				id='crimes-dropdown',
				options=[{'label': crime, 'value': crime} for crime in crimes.columns if crime != 'DISTRITOS'],
				value='POR CONSUMO DE DROGAS'
			),
			html.Div(id='crimes-graphs')
		])
	])
])

@app.callback(
	Output('crimes-graphs', 'children'),
	Input('crimes-dropdown', 'value')
)
def crimes_graphs(crimen):
	graphs = cv.get_all_graphs()
	map = cv.get_map(crimen)
	map_html = map._repr_html_()

	cloro_map = cv.get_map_with_cloro(crimen)
	cloro_map_html = cloro_map._repr_html_()

	return html.Div([
		graphs,
		html.Iframe(
			srcDoc=map_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		),
		html.Iframe(
			srcDoc=cloro_map_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		)
	])

@app.callback(
	Output('metro-graphs', 'children'),
	Input('metro-graphs', 'id')
)
def metro_graphs(_):
	metro_map_borders = mv.get_metro_map(cloropleth=False)
	metro_map_borders_html = metro_map_borders._repr_html_()

	metro_map_cloro = mv.get_metro_map(cloropleth=True)
	metro_map_cloro_html = metro_map_cloro._repr_html_()

	return html.Div([
		html.Iframe(
			srcDoc=metro_map_borders_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		),
		html.Iframe(
			srcDoc=metro_map_cloro_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		)
	])

@app.callback(
	Output('distrito-graph-cloropleth', 'children'),
	Input('distrito-dropdown-cloropleth', 'value')
)
def district_graphs(distrito):
	district_info = dv.get_district_info(distrito)
	district_cloropleth = dv.get_district_cloropleth(distrito)
	heatmap = dv.get_district_heatmap(distrito)

	choropleth_html = district_cloropleth._repr_html_()
	heatmap_html = heatmap._repr_html_()
	
	return html.Div([
		district_info,
		html.Iframe(
			srcDoc=choropleth_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		),
		html.Iframe(
			srcDoc=heatmap_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		)
	])

@app.callback(
	Output('general-graphs', 'children'),
	Input('general-graphs', 'id')
)
def general_graphs(_):
	graphs = gv.get_all_graphs()
	cloropleth_map = gv.get_madrid_cloropleth()
	heatmap = gv.get_madrid_heatmap()

	choropleth_html = cloropleth_map._repr_html_()
	heatmap_html = heatmap._repr_html_()
	
	return html.Div([
		graphs,
		html.Iframe(
			srcDoc=choropleth_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		),
		html.Iframe(
			srcDoc=heatmap_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		)
	])

if __name__ == '__main__':
	app.run_server(debug=True)

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[5], line 44, in crimes_graphs(crimen='POR CONSUMO DE DROGAS')
     39 @app.callback(
     40 	Output('crimes-graphs', 'children'),
     41 	Input('crimes-dropdown', 'value')
     42 )
     43 def crimes_graphs(crimen):
---> 44 	graphs = cv.get_all_graphs()
        cv = <handlers.crime_visualization.CrimeVisualization object at 0x00000193D1D08980>
     45 	map = cv.get_map(crimen)
     46 	map_html = map._repr_html_()

File g:\Mi unidad\Documents\1 Projects\MDS PRVD Proyecto visualizacion\PRVD-2\refactor\handlers\crime_visualization.py:19, in CrimeVisualization.get_all_graphs(
    self=<handlers.crime_visualization.CrimeVisualization object>
)
     17 correlation_matrix = numeric_columns.corr()
     18 fig_corr = px.imshow(correlation_matrix, text_auto=True, title="Matriz de Correlación entre Tipos de Incidentes")
---> 19 fig_cor